# SQL Alchemy INTRO Notebook

Introdução ao SQLAlchemy.

Baseada em:
[documentação] (https://docs.sqlalchemy.org/en/14/orm/tutorial.html#version-check)

In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base,sessionmaker
from sqlalchemy import Column, Integer, String



# - Criando uma engine.
engine = create_engine('sqlite:///sdata.db', echo=True)
Base = declarative_base()


In [25]:
print(Base)

<class 'sqlalchemy.orm.decl_api.Base'>


# Declarative Base

Constroi uma classe base para definições de classes declarativas.

A nova classe base receberá uma metaclasse que produz objetos Table apropriados e faz as chamadas mapper () apropriadas com base nas informações fornecidas declarativamente na classe e em quaisquer subclasses da classe.

No caso Abaixo, a nossa classe User vai receber uma nova declarative base, relacionada a Base acima.

In [2]:
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name, self.fullname, self.nickname)


In [26]:
print(User)

<class '__main__.User'>


In [3]:
User.__table__ 

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [4]:

Base.metadata.create_all(engine)


2021-12-02 11:19:50,618 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-02 11:19:50,623 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2021-12-02 11:19:50,629 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-02 11:19:50,633 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
Session = sessionmaker(bind=engine)
Session.configure(bind=engine)
session = Session()

In [6]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)

In [7]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')])


In [8]:
ed_user.nickname = 'eddie'

In [9]:
session.dirty

IdentitySet([])

In [10]:
session.commit()

2021-12-02 11:19:51,042 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-02 11:19:51,047 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-12-02 11:19:51,050 INFO sqlalchemy.engine.Engine [generated in 0.00384s] ('ed', 'Ed Jones', 'eddie')
2021-12-02 11:19:51,066 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-12-02 11:19:51,069 INFO sqlalchemy.engine.Engine [cached since 0.02232s ago] ('wendy', 'Wendy Williams', 'windy')
2021-12-02 11:19:51,073 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-12-02 11:19:51,077 INFO sqlalchemy.engine.Engine [cached since 0.0309s ago] ('mary', 'Mary Contrary', 'mary')
2021-12-02 11:19:51,086 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2021-12-02 11:19:51,088 INFO sqlalchemy.engine.Engine [cached since 0.04203s ago] ('fred', 'Fred Flintstone', 'freddy')
2021-

## Updating 

Para modificar valores, selecionar uma instancia e modificar o valor

In [11]:
ed_user.name='Eduardo'

Não esquecer de fazer o commit!

In [12]:
session.commit()

2021-12-02 11:19:51,832 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-02 11:19:51,841 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2021-12-02 11:19:51,842 INFO sqlalchemy.engine.Engine [generated in 0.00196s] (9,)
2021-12-02 11:19:51,850 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.id = ?
2021-12-02 11:19:51,851 INFO sqlalchemy.engine.Engine [generated in 0.00158s] ('Eduardo', 9)
2021-12-02 11:19:51,862 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
session.flush()

In [14]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')

In [15]:
session.add(fake_user)

In [28]:
s = session.query(User).filter(User.name.in_(['Eduardo', 'fakeuser'])).first()


2021-12-02 11:24:29,562 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
 LIMIT ? OFFSET ?
2021-12-02 11:24:29,566 INFO sqlalchemy.engine.Engine [cached since 267.8s ago] ('Eduardo', 'fakeuser', 1, 0)


In [31]:
print(s)
print(type(s))

<User(name='Eduardo', fullname='Ed Jones', nickname='eddie')>
<class '__main__.User'>


In [17]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2021-12-02 11:19:53,815 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2021-12-02 11:19:53,817 INFO sqlalchemy.engine.Engine [cached since 0.5644s ago] ('Edwardo', 'fakeuser')


[<User(name='fakeuser', fullname='Invalid', nickname='12345')>]

In [18]:
session.rollback()

2021-12-02 11:19:54,277 INFO sqlalchemy.engine.Engine ROLLBACK


In [19]:
ed_user.name

2021-12-02 11:19:54,480 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-02 11:19:54,484 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2021-12-02 11:19:54,486 INFO sqlalchemy.engine.Engine [generated in 0.00211s] (9,)


'Eduardo'

In [20]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)
query.all()

2021-12-02 11:19:54,608 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2021-12-02 11:19:54,614 INFO sqlalchemy.engine.Engine [generated in 0.00516s] ('%ed',)


[<User(name='fred', fullname='Fred Flintstone', nickname='freddy')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]